In [3]:
# Configure the Transformer layer
import os
BASE_FOLDER = '/Users/oblivisheee/Documents/GitHub/novelsdreamer-ru-t4m/'
os.chdir(BASE_FOLDER)
import tensorflow as tf
from modules.transformer_custom import Transformer
from modules.create_config import transformer_config, trainer_config, metrics_config
from modules.regularization import RegularizedDenseLayer
from hardware.parallel_compile import parallel_compile
config = transformer_config()
config_train = trainer_config()
config_metrics = metrics_config()
with parallel_compile(num_cores=4):
    embedding_layer = tf.keras.layers.Embedding(config["input_vocab_size"], config["d_model"])

    model = Transformer(config["num_layers"], config["d_model"], config["num_heads"], config["dff"],
                            config["input_vocab_size"], config["input_vocab_size"], pe_input=config["maximum_position_encoding"],
                            pe_target=config["maximum_position_encoding"], rate=config["dropout_rate"], embedding=embedding_layer)

    regularized_layer = RegularizedDenseLayer(config["d_model"])

    loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction=config_train['loss_reduction'])
    optimizer = tf.keras.optimizers.legacy.Adam(learning_rate=config_train['learning_rate'])

    metrics = [tf.keras.metrics.SparseCategoricalAccuracy(name=config_metrics['accuracy_set']), 
            tf.keras.metrics.MeanSquaredError(name=config_metrics['mean_sq_error']), 
            tf.keras.metrics.Precision(thresholds=0.5, name=config_metrics['precision']),]

    model.compile(optimizer=optimizer, loss=loss_object, metrics=metrics)

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0', '/job:localhost/replica:0/task:0/device:CPU:1', '/job:localhost/replica:0/task:0/device:CPU:2', '/job:localhost/replica:0/task:0/device:CPU:3')


In [ ]:
import os
BASE_FOLDER = '/Users/oblivisheee/Documents/GitHub/novelsdreamer-ru-t4m/'
os.chdir(BASE_FOLDER)
from modules.datagen import DataGenerator
MAIN_DATASET_DIR = 'dataset'
TRAIN_DATASET_DIR = os.path.join(MAIN_DATASET_DIR, 'train')
VALID_DATASET_DIR = os.path.join(MAIN_DATASET_DIR, 'valid')

datagen = DataGenerator(TRAIN_DATASET_DIR, VALID_DATASET_DIR)
(train_english, train_russian), (valid_english, valid_russian) = datagen.generate()